## Семинар 09: Нейросетевые модели поиска. Часть I.

В этом семинаре мы:
- изучим, как устроены простые эмбеддинги;
- обучим свои собственные эмбеддинги;
- научимся использовать предобученные из библиотеки [gensim](https://radimrehurek.com/gensim/auto_examples/index.html#documentation);
- решим задачу семантического матчинга с помощью DSSM.

In [ ]:
# %pip install --upgrade pip
# %pip install -r requirements.txt

In [2]:
import os
import numpy as np
import pandas as pd

from IPython.display import clear_output
from tqdm.notebook import tqdm

### Скачиваем и преобразуем данные

Нашим датасетом будет MS MARCO. Он содержит набор запросов (=сессий) и соответствующие пассажи (=документы).

Скачаем его с [huggingface](https://huggingface.co/datasets/Tevatron/msmarco-passage).

In [3]:
DATA_DIR = os.path.abspath("./data/")
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [4]:
from datasets import load_dataset

msmarco_dataset = load_dataset("Tevatron/msmarco-passage", split="train", cache_dir=DATA_DIR)
clear_output()
msmarco_dataset

Dataset({
    features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
    num_rows: 400782
})

Как видно из структуры датасета, для каждого запроса есть набор релевантных и нерелевантных пассажей. Посмотрим внимательнее на первую сессию и приведем его к удобному виду.

In [5]:
msmarco_dataset[0]

{'query_id': '1000094',
 'query': 'where is whitemarsh island',
 'positive_passages': [{'docid': '5399011',
   'title': 'Whitemarsh Island, Georgia',
   'text': 'Whitemarsh Island, Georgia. Whitemarsh Island (pronounced WIT-marsh) is a census-designated place (CDP) in Chatham County, Georgia, United States. The population was 6,792 at the 2010 census. It is part of the Savannah Metropolitan Statistical Area. The communities of Whitemarsh Island are a relatively affluent suburb of Savannah.'}],
 'negative_passages': [{'docid': '2670040',
   'title': 'What military strategy was used in the pacific?',
   'text': 'the strategy of island hopping was used by the United States in the Pacific theater of world war two. Thought of by Douglas MacArthur, island hopping was a strategy that used the technique of jumping from island to island on a chain to control the chain as a whole vs attacking all the islands at once.'},
  {'docid': '4683145',
   'title': 'Whakaari / White Island',
   'text': 'Fo

In [6]:
def convert_dataset_to_pandas(dataset):
    rows = []
    for qid, row in tqdm(enumerate(dataset), total=len(dataset)):
        query = row['query']
        for sample in row['positive_passages'] + row['negative_passages']:
            label = 1. if sample in row['positive_passages'] else 0.
            rows.append([qid, query, sample['text'], label])

    # Создаем DataFrame из списка rows
    df = pd.DataFrame(rows, columns=['qid', 'query', 'doc', 'label'])
    return df[['qid', 'query', 'doc', 'label']]


data = convert_dataset_to_pandas(msmarco_dataset)
data.shape

  0%|          | 0/400782 [00:00<?, ?it/s]

(12346948, 4)

Теперь нам нужно разделить текст на отдельные слова. Так как в тексте есть пунктуация, обычный str.split() использовать некорректно. Поэтому воспользуемся библиотекой `nltk`.

In [7]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

example = data["doc"][10][:105]
print(example)
print(tokenizer.tokenize(example))

Harbor Island - Physical Feature (Island) in Orange County. Harbor Island is a physical feature (island) 
['Harbor', 'Island', '-', 'Physical', 'Feature', '(', 'Island', ')', 'in', 'Orange', 'County', '.', 'Harbor', 'Island', 'is', 'a', 'physical', 'feature', '(', 'island', ')']


In [8]:
tqdm.pandas()

def tokenize_series(txts):
    assert isinstance(txts, pd.Series)
    # хак, чтобы токенизировать одно предложение один раз
    idx, _ = txts.factorize()
    txts_dict = dict(zip(idx, txts))
    for k, txt in tqdm(txts_dict.items(), leave=False):
        txts_dict[k] = tokenizer.tokenize(txt.lower())
    tokens = pd.Series(idx).progress_apply(lambda x: txts_dict[x])
    return tokens


data["query_tokens"] = tokenize_series(data["query"])
data["doc_tokens"] = tokenize_series(data["doc"])

  0%|          | 0/400782 [00:00<?, ?it/s]

  0%|          | 0/12346948 [00:00<?, ?it/s]

  0%|          | 0/5561835 [00:00<?, ?it/s]

  0%|          | 0/12346948 [00:00<?, ?it/s]

Посмотрим на получившийся датасет. Оказывается, что для каждого запроса в среднем 1 релевантный и 29 нерелевантных документов.

In [9]:
data.head()

,qid,query,doc,label,query_tokens,doc_tokens
0,0,where is whitemarsh island,"Whitemarsh Island, Georgia. Whitemarsh Island ...",1.0,"[where, is, whitemarsh, island]","[whitemarsh, island, ,, georgia, ., whitemarsh..."
1,0,where is whitemarsh island,the strategy of island hopping was used by the...,0.0,"[where, is, whitemarsh, island]","[the, strategy, of, island, hopping, was, used..."
2,0,where is whitemarsh island,"For the island near Dunedin, see White Island,...",0.0,"[where, is, whitemarsh, island]","[for, the, island, near, dunedin, ,, see, whit..."
3,0,where is whitemarsh island,"Jekyll Island, at 5,700 acres, is the smallest...",0.0,"[where, is, whitemarsh, island]","[jekyll, island, ,, at, 5, ,, 700, acres, ,, i..."
4,0,where is whitemarsh island,Sibu Island. A scuba diver at Sibu Island. Sib...,0.0,"[where, is, whitemarsh, island]","[sibu, island, ., a, scuba, diver, at, sibu, i..."


In [10]:
1.0 / data['label'].mean()

28.968417792647475

Разделим датасет на train / val / test. Разделяем с группировкой по сессиям (запросам).

In [11]:
TEST_SIZE=3_000
test_data = data[(400_000  - TEST_SIZE < data['qid']) & (data['qid'] <= 400_000)]
val_data = data[(400_000 - 2 * TEST_SIZE < data['qid']) & (data['qid'] <= 400_000 - TEST_SIZE)]
train_data = data[data['qid'] <= 400_000 - 2 * TEST_SIZE]

In [12]:
# Соберем токены и тексты для экспериментов.

query_texts, query_tokens = train_data.drop_duplicates(subset=["query"])[["query", "query_tokens"]].values.T
doc_texts, doc_tokens = train_data.drop_duplicates(subset=["doc"])[["doc", "doc_tokens"]].values.T

train_tokens = np.hstack([query_tokens, doc_tokens])
train_texts = np.hstack([query_texts, doc_texts])

### Эмбеддинги слов

#### Наивные подходы

Первая идея, как можно перевести последовательность слов в числовой вектор -- это one-hot кодирование.

На i-ом месте вектора стоит 1, если i-ое слово встречается в последовательности.

Небольшим апгрейдом этого решения будет следующее: на i-ое место ставим счетчик вхождений слова в последовательность.

Наконец коллекцию документов можно перевести вектор, используя tf-idf значения. Попробуем применить идею со счетчиками к нашей коллекции.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Так как тексты уже токенизированы, необходимо отключить _analyzer_.
cnt_vectorizer = CountVectorizer(analyzer=lambda x: x)

# Возьмем 1k документов из трейна.
corpus_cnt_embeddings = cnt_vectorizer.fit_transform(doc_tokens[:1000])
corpus_cnt_embeddings

<1000x8792 sparse matrix of type '<class 'numpy.int64'>'
	with 41967 stored elements in Compressed Sparse Row format>

Заметим, что уже для первых 1000 документов словарь у такой эмбеддер-модели разрастается до ~9 тысяч слов. При этом важность осмысленных токенов равна важности явно мусорных.

В результате мы получаем очень разреженную матрицу, в которой доля ненулевых значений меньше 1%. Если запустить на всем тренировочном корпусе, доля станет еще меньше.

In [14]:
vocabulary = cnt_vectorizer.get_feature_names_out()
print(f"{len(vocabulary)=}")
cnt_vectorizer.get_feature_names_out()[1000:]

len(vocabulary)=8792


array(['arezzo', 'arfura', 'argentina', ..., 'ð½ð', 'ð¾ð', 'ñ'],
      dtype=object)

In [15]:
print(corpus_cnt_embeddings.size / np.prod(corpus_cnt_embeddings.shape))

0.004773316651501365


Можно использовать уже такие эмбеддинги слов для своей задачи, но качество будет низким.

#### Prediction-based

Как уже сказано в лекции, есть разные prediction-based эмбеддинги: Word2Vec, GloVe, FastText, ELMo.

Есть удобная библиотека `gensim` с основными моделями, которые можно обучать.

Обучим свои Word2Vec эмбеддинги на наших текстах.

In [16]:
%%time

from gensim.models import Word2Vec

# Обучение может занять много времени.
# Используем 1/6 корпуса для экономии времени.
np.random.seed(42)
train_tokens_sample = np.random.choice(train_tokens, size=1_000_000)

model_w2v = Word2Vec(
    train_tokens_sample,
    vector_size=32, # размерность вектора-эмбеддинга
    min_count=5,    # рассматриваем слова, которые встречались от 5 раз
    window=5)       # ширина контекста

CPU times: user 7min 18s, sys: 8.19 s, total: 7min 26s
Wall time: 2min 38s


In [17]:
# Итак, мы получили эмбеддер, который дает осмысленные вектора для слов в корпусе!
model_w2v.wv.get_vector('cheese')

array([-1.1055639 ,  0.4345755 , -2.6897335 , -1.1729171 , -5.8404417 ,
       -5.037265  ,  1.7907585 ,  0.26450628, -2.3082716 , -3.5358524 ,
        4.869295  , -1.6566378 ,  1.8674453 ,  1.2612367 , -0.52317   ,
        1.5790911 ,  3.9923882 ,  1.7084982 ,  1.1043274 , -4.064599  ,
        2.8411613 , -4.6787996 ,  3.3800254 , -2.2277813 , -0.32656708,
        5.141197  , -5.9589806 , -0.69349086, -0.5008428 , -1.4719055 ,
        0.2566754 , -1.2377486 ], dtype=float32)

In [18]:
# На основе этого метода уже можно построить простой семантический поиск :)
model_w2v.wv.most_similar('cheese')

[('mozzarella', 0.9198249578475952),
 ('quinoa', 0.9128850698471069),
 ('ricotta', 0.9097282290458679),
 ('toppings', 0.9076871275901794),
 ('crackers', 0.9058236479759216),
 ('bacon', 0.9055699706077576),
 ('hummus', 0.9045976996421814),
 ('rice', 0.8967082500457764),
 ('pepperoni', 0.8940117359161377),
 ('macaroni', 0.8871335387229919)]

In [19]:
# Получим ошибку для слова, которого нет в корпусе :|
# model_w2v.wv.get_vector("king00")

#### Pre-trained

Чтобы получить действительно качественные эмбеддинги, используют большие корпуса текстов: Wiki, Common Crawl, C4 и другие.

И обучение значительно больше времени!

Благодаря бОльшему размеру датасетов, удается "вложить" больше контекстов в компактные вектора.

Свои эмбеддинги с нуля обучают редко. Вместо этого используются предобученные модели.

Посмотрим, какие модели есть в `gensim`:

In [20]:
# # Если ячейка ниже не сработает, надо скачать модели вручную:
# !source download_data.sh https://cloud.mail.ru/public/UwxC/GitCjd7Zr ./data/gensim-data.zip
# !unzip -d ~/gensim-data ./data/gensim-data.zip

In [21]:
import gensim.downloader as api

print('\n'.join(api.info()['models'].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [22]:
%%time

model_ft = api.load("fasttext-wiki-news-subwords-300")
clear_output()

CPU times: user 2min 56s, sys: 2.9 s, total: 2min 59s
Wall time: 2min 59s


In [23]:
model_ft.get_vector("cheese")[:32]

array([-0.037223  ,  0.01457   , -0.054043  , -0.048354  , -0.032754  ,
       -0.09059   ,  0.037491  , -0.095673  ,  0.0093863 , -0.0047703 ,
       -0.022836  ,  0.027393  ,  0.039528  , -0.067544  , -0.029422  ,
        0.01189   ,  0.074464  ,  0.01498   ,  0.019062  , -0.035061  ,
        0.0042614 , -0.024094  , -0.041069  ,  0.0252    ,  0.036754  ,
        0.063787  ,  0.00034417,  0.14274   ,  0.054383  ,  0.031954  ,
        0.0016913 , -0.044631  ], dtype=float32)

In [24]:
model_ft.get_vector("cheese").shape

(300,)

In [25]:
# В пространстве эмбеддингов наблюдается линейность!
model_ft.most_similar(positive=["king", "woman"], negative=["man"])

[('queen', 0.7786749005317688),
 ('queen-mother', 0.7143871784210205),
 ('king-', 0.6981282234191895),
 ('queen-consort', 0.6724597811698914),
 ('monarch', 0.6666999459266663),
 ('child-king', 0.6663159132003784),
 ('boy-king', 0.660534679889679),
 ('princess', 0.653827428817749),
 ('ex-queen', 0.652145504951477),
 ('kings', 0.6497675180435181)]

In [26]:
# И синтаксическая тоже!
model_ft.most_similar(positive=["cats", "dog"], negative=["cat"])

[('dogs', 0.9464988112449646),
 ('dogs-', 0.7996147871017456),
 ('puppies', 0.7778717279434204),
 ('dog-owners', 0.7610784769058228),
 ('pooches', 0.7494959831237793),
 ('labradors', 0.7411291003227234),
 ('beagles', 0.7399105429649353),
 ('dachshunds', 0.7363396286964417),
 ('pets', 0.730984628200531),
 ('canines', 0.7275385856628418)]

#### Визуализация

Нарисуем получившиеся эмбеддинги, чтобы дополнительно убедиться в их осмысленности.

Сейчас векторы имеют размерности больше 30. Используем методы понижения размерности, например, t-sne, чтобы нарисовать их.

In [27]:
# Ограничим словари моделей до 3000 слов.
words = {}
words["model_w2v"] = model_w2v.wv.index_to_key[:3000]
words["model_ft"] = model_ft.index_to_key[:3000]

word_vectors = {}
word_vectors["model_w2v"] = np.array([model_w2v.wv.get_vector(w) for w in words["model_w2v"]])
word_vectors["model_ft"] = np.array([model_ft.get_vector(w) for w in words["model_ft"]])

In [28]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook
from sklearn.manifold import TSNE

output_notebook()

def draw_vectors(words_dict, vectors_dict, color="green"):
    word_tsne = TSNE(2).fit_transform(vectors_dict)
    word_tsne = (word_tsne - word_tsne.mean(axis=0)) / word_tsne.std(axis=0)

    x, y = word_tsne[:, 0], word_tsne[:, 1]
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({'x': x, 'y': y, 'color': color, 'token': words_dict})

    fig = pl.figure(active_scroll='wheel_zoom', width=600, height=400)
    fig.scatter('x', 'y', size=10, color='color', alpha=0.25, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[("token", "@token")]))
    pl.show(fig)


Loading BokehJS ...

In [29]:
# Обученные Word2Vec эмбеддинги слов.
draw_vectors(words["model_w2v"], word_vectors["model_w2v"])

In [30]:
# Предобученные FastText эмбеддинги слов.
draw_vectors(words["model_ft"], word_vectors["model_ft"])

### Эмбеддинги предложений

#### Наивный подход

Эмбеддинги отдельных слов можно использовать для получения эмбеддингов _коротких_ предложений.

Простым подходом будет усреднение эмбеддингов слов. Применим этот метод на подмножестве запросов.

In [31]:
# Нарисуем усредненные эмбеддинги запросов. Ограничимся 3000 случайными запросами.
idx = np.random.randint(query_texts.shape[0], size=3000)

In [32]:
# Обученные Word2Vec эмбеддинги запросов.
sentence_vectors_w2v = np.array([model_w2v.wv.get_mean_vector(sentence) for sentence in query_tokens[idx]])
draw_vectors(query_texts[idx], sentence_vectors_w2v)

In [33]:
# Предобученные FastText эмбеддинги запросов.
sentence_vectors_ft = np.array([model_ft.get_mean_vector(sentence) for sentence in query_tokens[idx]])
draw_vectors(query_texts[idx], sentence_vectors_ft)

#### Doc2Vec

Вместо агрегации векторов слов можно обучить эмбеддинги сразу для текстов.

Исследуем, как сработает Doc2Vec для нашей коллекции!

In [34]:
%%time

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Ограничимся 50_000 первых документов.
idx = np.arange(50_000)
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(doc_tokens[idx])]
model_doc2vec = Doc2Vec(documents, vector_size=32, window=5, min_count=5, workers=4)

CPU times: user 1min 45s, sys: 13 s, total: 1min 58s
Wall time: 1min 3s


In [35]:
# Считаем вектор для запроса на основе выученных эмбеддингов слов внутри Doc2Vec.
qid = 10
vector = model_doc2vec.infer_vector(query_tokens[qid], epochs=1000)

# Рассчитываем косинусную близость между эмбеддингами документов и вектором запроса и берем topn.
sims = model_doc2vec.dv.most_similar([vector], topn=5)
print(f'Query =\t\t"{query_texts[qid]}"')
for i, (index, label) in enumerate(sims):
    print(f"Top {i + 1} doc:\t", doc_texts[idx][index])

Query =		"where was the movie goonies filmed"
Top 1 doc:	 To share a link, enter the URL into the share menu at the top of your Timeline or homepage. You can include a message next to your link if you like. Be sure to set privacy before you post, then click Post. While you're browsing the web, you may also see opportunities to post links back to Facebook.Clicking a Like or Recommend button on another website can create a story for you on Facebook.ou can also choose who can see your post. Keep in mind that when you tag someone in a photo or post, that photo or post may also be shared with that person and their friends. Learn how to turn this setting off.
Top 2 doc:	 Blarney Castle, 1954. The castle originally dates from before 1200, when a wooden structure was believed to have been built on the site, although no evidence remains of this. Around 1210 this was replaced by a stone fortification.
Top 3 doc:	 DMV is a song by the rock band Primus. Interscope Records asked Primus to release t

Нюансы:
* Наблюдаемое качество оставляет желать лучшего. Но это ожидаемо, ведь у нас всего 1 позитивный пример на 29 негативных! Можно попробовать обучить модель только на позитивных примерах.
* Все рассмотренные методы обучения эмбеддингов относятся к self-supervised обучению, так как метки релевантности не используются. Supervised методы мы рассмотрим в следующих лекциях.
* Чтобы добавить новый документ в коллекцию, нужно обучить модель заново. То есть замерить качество для неизвестных документов без переобучения не получится.
* Аналогично примеру выше, можно обучить модель не на тексты документов, а на тексты запросов. По сравнению с простым усреднением эмбеддингов слов качество будет выше. Так можно будет искать похожие запросы и добавлять кандидатов в выдачу для верхнего ранжирования.

### Семантический поиск

Наконец, вернемся к задаче поиска. Ограничимся топ-10 документами в выдаче.

Требуется отранжировать пассажи для данного запроса, чтобы релевантный пассаж стоял выше нерелевантных.

#### Метрика

Чтобы оценить качество , будем смотреть на метрику [Mean Reciprocal Rank](https://www.evidentlyai.com/ranking-metrics/mean-reciprocal-rank-mrr) (MRR). Она определяется так:

$$ MRR = \frac{1}{|Q|} \sum_{q_i} \frac{1}{rank_{i}},$$

где $ rank_i $ - позиция __первого релевантного__ док-та для запроса $q_i$, $ |Q| $ - кол-во запросов в выборке.

In [36]:
import torch
from torchmetrics.retrieval import RetrievalMRR

# Подробнее про реализацию:
# https://github.com/Lightning-AI/torchmetrics/blob/master/src/torchmetrics/functional/retrieval/reciprocal_rank.py
mrr = RetrievalMRR(top_k=10)

def MRR(preds, target, qids):
    assert isinstance(preds, np.ndarray)
    assert isinstance(target, np.ndarray)
    assert isinstance(qids, np.ndarray)
    score = mrr(torch.Tensor(preds), torch.Tensor(target), indexes=torch.LongTensor(qids - min(qids)))
    return score.item()

In [37]:
results = {}

test_doc_texts, test_doc_tokens = test_data[["doc", "doc_tokens"]].values.T
test_query_texts, test_query_tokens = test_data.drop_duplicates(subset=["query"])[["query", "query_tokens"]].values.T
test_query_idx = test_data["qid"].factorize()[0] # индексы сессий понадобятся ниже

#### Бейзлайн: Random

Измерим качество случайного предсказания релевантности:

In [38]:
np.random.seed(42)
results["random"] =  MRR(np.random.random(len(test_data)), test_data['label'].values, test_data['qid'].values)

In [39]:
results["random"]

0.09801560640335083

#### Бейзлайн: BM25

Теперь применим алгоритм BM25. До появления трансформеров это был стабильно хороший бейзлайн в задаче ранжирования.

In [40]:
from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(list(test_doc_tokens))

In [41]:
%%time
# Ячейка исполняется порядка 10 минут!

bm25_preds = np.zeros(len(test_data))
for q_text, q_tokens in tqdm(zip(test_query_texts, test_query_tokens), total=len(test_query_texts)):
    doc_scores = bm25.get_scores(q_tokens)
    mask = test_data['query'] == q_text
    bm25_preds[mask] = doc_scores[mask]

  0%|          | 0/3000 [00:00<?, ?it/s]

CPU times: user 10min 24s, sys: 6.13 s, total: 10min 30s
Wall time: 10min 28s


In [ ]:
%%time
# Распараллелим!

import multiprocessing as mp

def get_bm25_scores(args):
    q_text, q_tokens = args
    doc_scores = bm25.get_scores(q_tokens)
    mask = test_data['query'] == q_text
    return np.where(mask, doc_scores, 0)


cpu_count = mp.cpu_count()
gen = zip(test_query_texts, test_query_tokens)
with mp.Pool(cpu_count) as pool:
    doc_scores = list(tqdm(pool.imap(get_bm25_scores, gen), total=len(test_query_texts)))
doc_scores = np.sum(doc_scores, axis=0)

In [43]:
assert np.allclose(doc_scores, bm25_preds)
assert (doc_scores == bm25_preds).all()

In [44]:
results["bm25"] = MRR(bm25_preds, test_data['label'].values, test_data['qid'].values)

In [45]:
results["bm25"]

0.6004154682159424

#### DSSM-like

Теперь попробуем решить задачу с помощью наших и предобученных эмбеддингов.

Посчитаем эмбеддинги документов и запросов, усреднив эмбеддинги слов в них, и затем нормируем их.

Полученные эмбеддинги документов отранжируем по косинусной близости отдельно для каждого запроса.

In [46]:
def get_embedder_model_preds(model, doc_tokens, query_tokens, test_query_idx=None):
    # Усредняем эмбеддинги слов в документе и нормализуем
    doc_embeddings = np.stack([model.get_mean_vector(tokens) for tokens in tqdm(doc_tokens, leave=False)])
    doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=-1, keepdims=True)

    # Усредняем эмбеддинги слов в запросе и нормализуем
    query_embeddings = np.stack([model.get_mean_vector(tokens) for tokens in tqdm(query_tokens, leave=False)])
    query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=-1, keepdims=True)
    query_embeddings = query_embeddings[test_query_idx]

    scores = (query_embeddings * doc_embeddings).sum(axis=-1)
    return scores

In [47]:
model_w2v_preds = get_embedder_model_preds(model_w2v.wv, test_doc_tokens, test_query_tokens, test_query_idx)
model_ft_preds = get_embedder_model_preds(model_ft, test_doc_tokens, test_query_tokens, test_query_idx)

results["word2vec"] = MRR(model_w2v_preds, test_data['label'].values, test_data['qid'].values)
results["fasttext"] = MRR(model_ft_preds, test_data['label'].values, test_data['qid'].values)

  0%|          | 0/92658 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/92658 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

#### Сравниваем результаты

In [48]:
for name, value in sorted(results.items(), key=lambda x: -x[1]):
    print(f'{value:.5f}\t', name)

0.60042	 bm25
0.24425	 fasttext
0.21940	 word2vec
0.09802	 random


Уже с помощью bm25 удалось кратно превысить рандомное предсказание.

А эмбеддинги показали сильно более низкое качество!

При этом предобученный FastText сравним с обученным нами Word2Vec на небольшой части корпуса.

Попробуем побить этот результат в следующем семинаре. Stay tuned!

__Что можно еще попробовать:__
* Обучить Word2Vec на всем тренировочном корпусе текстов: включая запросы и без них.
* Обучить Doc2Vec на всем корпусе и ранжировать с помощью него.
* Попробовать усреднять эмбеддинги слов с весами tf-idf.
* Смешать предикты эмбеддинговых моделей и BM25 и обучить ансамбль из моделей.
* Обучить ранжирующую модель поверх агрегированных наивных или count-based эмбеддингов.
* Использовать более продвинутые методы: USE, ELMo и другие.